# Classical Simulation of Quantum Circuits

Quantum computation involves evolving an initial state via a sequence of operations (gates) and measuring the result. While the ultimate goal is execution on a real device, classical simulation remains crucial, especially in early design stages.  

Classical simulations are essential when suitable quantum hardware is unavailable or limited in scale, depth, or accuracy. Even as hardware improves, simulations allow access to _all_ amplitudes of a quantum state—unlike real devices, which only provide probabilistic measurement outcomes. They also support studies of quantum error correction and provide baselines to assess quantum advantage over classical computation.

Typically, classical simulation uses consecutive matrix-vector multiplications, storing the full state vector or employing tensor network methods. This quickly becomes intractable due to the exponential growth of the state with qubit number. Decision diagram-based methods offer a promising alternative by exploiting redundancies to reduce memory requirements.

The _MQT_ offers the classical quantum circuit simulator _DDSIM_ that can be used to perform various quantum circuit simulation tasks based on using decision diagrams as a data structure. This includes strong and weak simulation, approximation techniques, noise-aware simulation, hybrid Schrödinger-Feynman techniques, support for dynamic circuits, the computation of expectation values, the simulation of mixed-dimensional systems, and more.

In this tutorial, we'll use Qiskit to define quantum circuits and simulate them using:
- **MQT DDSIM**: A decision-diagram-based simulator from the Munich Quantum Toolkit
- **Qiskit Statevector Sampler**: IBM's built-in state vector simulator

We'll compare their performance on various quantum circuits.

## Simulating Simple Quantum Gates

Let's start with the most basic quantum operation: the X gate (NOT gate).

In [ ]:
from qiskit import QuantumCircuit

# Create a 1-qubit circuit with an X gate
circ = QuantumCircuit(1)
circ.x(0)
circ.draw(output="mpl")

In [ ]:
from mqt.ddsim import DDSIMProvider
from qiskit.visualization import plot_histogram

# Get DDSIM backend and run simulation
provider = DDSIMProvider()
backend = provider.get_backend("qasm_simulator")
result = backend.run(circ, shots=10000).result()

# Visualize results
plot_histogram(result.get_counts())

## Simulating a CNOT Gate

The CNOT (Controlled-NOT) is a fundamental two-qubit entangling gate.

In [ ]:
# Create a 2-qubit circuit: X gate on qubit 0, then CNOT(0→1)
circ = QuantumCircuit(2)
circ.x(0)
circ.cx(0, 1)
circ.measure_all()
circ.draw(output="mpl")

In [ ]:
# Simulate and visualize
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Creating a Bell State

The Bell state is a maximally entangled two-qubit state: (|00⟩ + |11⟩) / √2

We create it using a Hadamard gate followed by a CNOT.

In [ ]:
# Start with a Hadamard gate to create superposition
circ = QuantumCircuit(2)
circ.h(0)
circ.draw(output="mpl")

In [ ]:
# After Hadamard: equal probability of |0⟩ and |1⟩
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

In [ ]:
# Add CNOT to create entanglement
circ.cx(0, 1)
circ.measure_all()
circ.draw(output="mpl")

In [ ]:
# Bell state: 50% |00⟩, 50% |11⟩ (perfectly correlated)
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Creating a GHZ State

The GHZ (Greenberger-Horne-Zeilinger) state extends the Bell state to three qubits: (|000⟩ + |111⟩) / √2

In [ ]:
# Create 3-qubit GHZ state
circ = QuantumCircuit(3)
circ.h(2)           # Superposition on qubit 2
circ.cx(2, 1)       # Entangle qubit 2 with qubit 1
circ.cx(1, 0)       # Entangle qubit 1 with qubit 0
circ.measure_all()
circ.draw(output="mpl")

In [ ]:
# GHZ state: 50% |000⟩, 50% |111⟩
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

# Using MQT Bench for Standard Circuits

MQT Bench provides a collection of benchmark quantum circuits at different abstraction levels.

**Documentation**: https://mqt.readthedocs.io/projects/bench/en/latest/Benchmark_selection.html

In [ ]:
from mqt.bench import get_benchmark, BenchmarkLevel

# Load a 3-qubit GHZ circuit from MQT Bench
circ = get_benchmark(benchmark="ghz", level=BenchmarkLevel.ALG, circuit_size=3)
circ.draw(output="mpl")

## Quantum Phase Estimation Example

Quantum Phase Estimation (QPE) is a fundamental algorithm for finding eigenvalues of unitary operators.

In [ ]:
# Load and simulate a 5-qubit QPE circuit
circ = get_benchmark(benchmark="qpeinexact", level=BenchmarkLevel.ALG, circuit_size=5)
circ.draw(output="mpl")

In [ ]:
result = backend.run(circ, shots=10000).result()
plot_histogram(result.get_counts())

## Performance Comparison: Qiskit vs DDSIM

Let's benchmark both simulators on increasingly large GHZ circuits.

### Qiskit StatevectorSampler Performance

In [ ]:
from qiskit.primitives import StatevectorSampler
import time

sampler = StatevectorSampler()
runtimes = []
value_range = range(10, 24, 2)

for i in value_range:
    # Get GHZ circuit of size i
    circ = get_benchmark(benchmark="ghz", level=BenchmarkLevel.ALG, circuit_size=i)
    circ.measure_all()
    
    # Measure execution time
    start = time.time()
    job = sampler.run([circ], shots=10000)
    result = job.result()
    runtime = time.time() - start
    
    runtimes.append(runtime)
    print(f"GHZ-{i}: {runtime:.6f}s")

In [ ]:
import matplotlib.pyplot as plt

# Plot Qiskit runtimes
plt.xlabel("Circuit Size (qubits)")
plt.ylabel("Runtime (s)")
plt.title("Qiskit StatevectorSampler Performance")
plt.plot(value_range, runtimes)
plt.grid(True, alpha=0.3)
plt.show()

### DDSIM Performance

In [ ]:
backend = provider.get_backend("qasm_simulator")
runtimes_ddsim = []

for i in value_range:
    # Get GHZ circuit of size i
    circ = get_benchmark(benchmark="ghz", level=BenchmarkLevel.ALG, circuit_size=i)
    circ.measure_all()
    
    # Run simulation (DDSIM tracks its own timing)
    result = backend.run(circ, shots=10000).result()
    runtimes_ddsim.append(result.time_taken)
    print(f"GHZ-{i}: {result.time_taken:.6f}s")

# Plot DDSIM runtimes
plt.xlabel("Circuit Size (qubits)")
plt.ylabel("Runtime (s)")
plt.title("DDSIM Performance")
plt.plot(value_range, runtimes_ddsim)
plt.grid(True, alpha=0.3)
plt.show()

### Side-by-Side Comparison

DDSIM's decision diagram approach often outperforms traditional state vector methods for circuits with high redundancy.

In [ ]:
plt.figure(figsize=(10, 6))
plt.xlabel("Circuit Size (qubits)")
plt.ylabel("Runtime (s)")
plt.title("Simulation Runtime Comparison: Qiskit vs DDSIM")
plt.plot(value_range, runtimes, marker='o', label="Qiskit StatevectorSampler")
plt.plot(value_range, runtimes_ddsim, marker='s', label="MQT DDSIM")
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## Exercises for Further Exploration

Try extending this tutorial with:

1. **Different Algorithms**: Choose other circuits from MQT Bench (e.g., `grover`, `qaoa`, `vqe`)
   - See: https://mqt.readthedocs.io/projects/bench/en/latest/Benchmark_selection.html

2. **Other Qiskit Primitives**: Explore `Estimator` for expectation values
   - See: https://docs.quantum.ibm.com/api/qiskit/primitives

3. **Advanced DDSIM Features**: Try different simulation modes (hybrid, stochastic)
   - See: https://mqt.readthedocs.io/projects/ddsim/en/latest/Simulators.html

4. **Performance Analysis**: Compare simulators on different circuit types (e.g., quantum volume, random circuits)